# Introduction 

The idea is to use LiftRsNumber.py to convert old rs number to new rs number, and use the data file b132_SNPChrPosOnRef_37_1.bcp.gz (a data file containing each dbSNP and its positions in NCBI build 37).  
1. Extract and lift rs numbers  
Use the tools LiftRsNumber.py to lift the rs number in the map file from old build to new build.
2. Lookup SNP positions from rs number  
dbSNP provides a file b132_SNPChrPosOnRef_37_1.bcp.gz which contains rsNumber, chromosome and its position. Use this file along with the new rsNumber obtained in the first step. In practice, some rs numbers do not exist in build 132, or not suitable to be considered ( e.g. they do not reside on human reference, or they are mapped to multiple locations, these scenarios are noted by the chromosome column with values like "AltOnly", "Multi", "NotOn", "PAR", "Un"), we can drop them in the liftover procedure. We will obtain the rs number and its position in the new build after this step.

In [1]:
import os
import io
import sys
import re
import json
import gzip
import shlex, subprocess
import pandas as pd
import numpy as np
# from joblib import Parallel, delayed

# Extract and lift rs numbers 

## Load SNP History and Archive files

In [2]:
%%time
RS_HISTORY = set()  # store rs

# record obsolete rs number
with gzip.open('/home/tun01458/lib/dbSNP/SNPHistory.bcp.gz', 'rb') as f_gz:
    f_history = io.BufferedReader(f_gz)  # speed up read
    for line in f_gz:
        line = line.decode('utf8')  # covert byte to string
        # print(line)
        fd = line.strip().split('\t')
        if line.find('re-activ') < 0:
            RS_HISTORY.add(fd[0])

CPU times: user 1min 13s, sys: 1.82 s, total: 1min 15s
Wall time: 1min 15s


In [3]:
%%time
RS_MERGE = dict()  # high_rs -> (lower_rs, current_rs)

# record rs number merge history
with gzip.open('/home/tun01458/lib/dbSNP/RsMergeArch.bcp.gz', 'rb') as f_gz:
    f_archive = io.BufferedReader(f_gz)  # speed up read
    for line in f_archive:
        line = line.decode('utf8')  # covert byte to string
        fd = line.strip().split('\t')
        h, l = fd[0], fd[1]
        c = fd[6]
        RS_MERGE[h] = (l, c)

CPU times: user 44.5 s, sys: 4.83 s, total: 49.4 s
Wall time: 49.4 s


## Collect rsNumbers

In [4]:
# all files
build36_genotype_filenames = os.listdir('user_data/build36_genotype/')
print(len(build36_genotype_filenames))
build36_genotype_filenames

66


['user123_file54_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user549_file252_yearofbirth_1948_sex_XY.23andme.txt',
 'user202_file85_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user146_file62_yearofbirth_1971_sex_XY.23andme.txt',
 'user64_file30_yearofbirth_1965_sex_XY.23andme.txt',
 'user595_file277_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user26_file10_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user22_file8_yearofbirth_1975_sex_XY.23andme.txt',
 'user14_file6_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user481_file215_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user468_file200_yearofbirth_1978_sex_XX.23andme.txt',
 'user347_file157_yearofbirth_1952_sex_XX.23andme.txt',
 'user296_file127_yearofbirth_unknown_sex_unknown.23andme.txt',
 'user865_file427_yearofbirth_1979_sex_XY.23andme.txt',
 'user75_file39_yearofbirth_1982_sex_XY.23andme.txt',
 'user60_file27_yearofbirth_1982_sex_XY.23andme.txt',
 'user55_file24_yearofbirth_unknown_sex_unknown.23andme.txt',
 'u

In [5]:
# union all rsID, not include SNP starting with 'i'
buid36_rs_union = set()
for i, filename in enumerate(build36_genotype_filenames):
    print('{}/{}'.format(i + 1, len(build36_genotype_filenames)), end='\r')
    build36_genotype = pd.read_csv(os.path.join('user_data/build36_genotype/',
                                                filename),
                                   sep='\t',
                                   header=None,
                                   usecols=[0],
                                   names=['rsid'],
                                   comment='#')

    # union
    buid36_rs_union = buid36_rs_union | set(
        [x[2:] for x in build36_genotype['rsid'] if x[0:2] == 'rs'])

print()
print(len(buid36_rs_union))

66/66
1266046


## Lift over rsNumber

In [6]:
RsNum_MAP = {}
for rsNum in buid36_rs_union:
    # print(rsNum)
    # rs number not appear in RS_MERGE -> there is no merge on this rs
    if rsNum not in RS_MERGE:
        # print(rsNum, rsNum, "unchanged")
        continue
    # lift rs number
    while True:
        if rsNum in RS_MERGE:
            rsLow, rsCurrent = RS_MERGE[rsNum]
            if rsCurrent not in RS_HISTORY:
                RsNum_MAP['rs' + str(rsNum)] = 'rs' + str(rsCurrent)
                # print(rsNum, rsCurrent, "lifted")
                break
            else:
                rsNum = rsLow
        else:
            # print(rsNum, rsNum, "unlifted" )
            break
            
print(len(RsNum_MAP))
RsNum_MAP

1134


{'rs28935178': 'rs132630275',
 'rs386420751': 'rs112184173',
 'rs11617079': 'rs12015',
 'rs4141926': 'rs3126628',
 'rs35774380': 'rs2750557',
 'rs10499304': 'rs9480047',
 'rs11069912': 'rs9515509',
 'rs28634043': 'rs9701850',
 'rs28934899': 'rs79931499',
 'rs28929500': 'rs137854479',
 'rs10851319': 'rs9523445',
 'rs750650420': 'rs606231234',
 'rs61726457': 'rs137852632',
 'rs6507641': 'rs16809',
 'rs12560648': 'rs9550658',
 'rs17382334': 'rs11540037',
 'rs66616071': 'rs66616070',
 'rs28937303': 'rs137852466',
 'rs28546178': 'rs5888257',
 'rs12818259': 'rs1966010',
 'rs4073952': 'rs4052427',
 'rs28929487': 'rs121917873',
 'rs17152154': 'rs2229862',
 'rs28930972': 'rs121908728',
 'rs10454556': 'rs9524587',
 'rs28935212': 'rs137852392',
 'rs35861491': 'rs34049890',
 'rs28358271': 'rs7520428',
 'rs12461010': 'rs679057',
 'rs28937288': 'rs137852441',
 'rs35964018': 'rs2522626',
 'rs4987310': 'rs2229569',
 'rs28491442': 'rs9783095',
 'rs200931747': 'rs193303033',
 'rs34787276': 'rs2755727',


# Update rs, chr, pos in build36 files

## Load dbSNP file

In [7]:
%%time
# dbSNP file which contains rsNumber, chromosome and its position
SNPChrPosOnRef = None

# Look up SNP positions from rs number
with gzip.open('/home/tun01458/lib/dbSNP/b132_SNPChrPosOnRef_37_1.bcp.gz',
               'rb') as f_gz:
    f_SNPChrPosOnRef = io.BufferedReader(f_gz)  # speed up read
    SNPChrPosOnRef = pd.read_csv(f_SNPChrPosOnRef,
                                 sep='\t',
                                 usecols=[0, 1, 2],
                                 names=['rsid', 'chr', 'pos'])

print(SNPChrPosOnRef.shape)
SNPChrPosOnRef.head()

(30443446, 3)
CPU times: user 17.9 s, sys: 2.46 s, total: 20.3 s
Wall time: 20.4 s


,rsid,chr,pos
0,3,13,32446841.0
1,4,13,32447221.0
2,5,7,91839109.0
3,6,7,91747130.0
4,7,7,91779556.0


In [8]:
# SNPChrPosOnRef['chr'].value_counts()

In [9]:
# SNPChrPosOnRef = SNPChrPosOnRef[(SNPChrPosOnRef['chr'] != 'Multi')
#                                 & (SNPChrPosOnRef['chr'] != 'X')
#                                 & (SNPChrPosOnRef['chr'] != 'Y')
#                                 & (SNPChrPosOnRef['chr'] != 'NotOn')
#                                 & (SNPChrPosOnRef['chr'] != 'Un')
#                                 & (SNPChrPosOnRef['chr'] != 'PAR')
#                                 & (SNPChrPosOnRef['chr'] != 'AltOnly')
#                                 & (SNPChrPosOnRef['chr'] != 'MT')]

In [10]:
# # remove missing value in pos
# SNPChrPosOnRef = SNPChrPosOnRef[~SNPChrPosOnRef['pos'].isna()]

In [11]:
# add prefix to rsid
SNPChrPosOnRef['rsid'] = 'rs' + SNPChrPosOnRef['rsid'].astype(str)

In [12]:
# set rsid as index
SNPChrPosOnRef = SNPChrPosOnRef.set_index(['rsid'])

In [13]:
SNPChrPosOnRef.head()

,chr,pos
rsid,,
rs3,13,32446841.0
rs4,13,32447221.0
rs5,7,91839109.0
rs6,7,91747130.0
rs7,7,91779556.0


## Update build36 file each file

In [14]:
%%time
# load genotype
for i, filename in enumerate(build36_genotype_filenames):
    print('{}/{}'.format(i + 1, len(build36_genotype_filenames)))
    print(filename)

    build36_genotype = pd.read_csv(os.path.join('user_data/build36_genotype/',
                                                filename),
                                   sep='\t',
                                   header=None,
                                   names=['rsid', 'chr', 'pos', 'genotype'],
                                   comment='#')

    # update rs number
    build36_genotype.replace({'rsid': RsNum_MAP})
    build36_genotype = build36_genotype.set_index(['rsid'])

    # update chr and pos
    build36_genotype.update(SNPChrPosOnRef)

    # convert pos dtypes as int32
    build36_genotype = build36_genotype.astype({'pos': 'int32'})
    build36_genotype['pos'] = build36_genotype['pos'] + 1

    # write to file
    comment = '''# This data was lifted from build36 to build37. Below is original information. 
# ********* Original Information ***********
# This data file generated by 23andMe at: Wed Nov  2 19:32:28 2011
#
# Below is a text version of your data. Fields are TAB-separated
# Each line corresponds to a single SNP.  For each SNP, we provide its identifier
# (an rsid or an internal id), its location on the reference human genome, and the
# genotype call oriented with respect to the plus strand on the human reference
# sequence.     We are using reference human assembly build 36.  Note that it is possible
# that data downloaded at different times may be different due to ongoing improvements
# in our ability to call genotypes. More information about these changes can be found at:
# https://www.23andme.com/you/download/revisions/
#
# More information on reference human assembly build 36:
# http://www.ncbi.nlm.nih.gov/projects/mapview/map_search.cgi?taxid=9606&build=36
#
# rsid	chromosome	position	genotype'''
    with open(os.path.join('user_data/lifted_build36_to_build37/', filename),
              'w') as f:
        f.write(comment + '\n')
        build36_genotype.to_csv(f, sep='\t', header=None)

1/66
user123_file54_yearofbirth_unknown_sex_unknown.23andme.txt


<string>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.


2/66
user549_file252_yearofbirth_1948_sex_XY.23andme.txt
3/66
user202_file85_yearofbirth_unknown_sex_unknown.23andme.txt
4/66
user146_file62_yearofbirth_1971_sex_XY.23andme.txt
5/66
user64_file30_yearofbirth_1965_sex_XY.23andme.txt
6/66
user595_file277_yearofbirth_unknown_sex_unknown.23andme.txt
7/66
user26_file10_yearofbirth_unknown_sex_unknown.23andme.txt
8/66
user22_file8_yearofbirth_1975_sex_XY.23andme.txt
9/66
user14_file6_yearofbirth_unknown_sex_unknown.23andme.txt
10/66
user481_file215_yearofbirth_unknown_sex_unknown.23andme.txt
11/66
user468_file200_yearofbirth_1978_sex_XX.23andme.txt
12/66
user347_file157_yearofbirth_1952_sex_XX.23andme.txt
13/66
user296_file127_yearofbirth_unknown_sex_unknown.23andme.txt
14/66
user865_file427_yearofbirth_1979_sex_XY.23andme.txt
15/66
user75_file39_yearofbirth_1982_sex_XY.23andme.txt
16/66
user60_file27_yearofbirth_1982_sex_XY.23andme.txt
17/66
user55_file24_yearofbirth_unknown_sex_unknown.23andme.txt
18/66
user279_file113_yearofbirth_unknown_

In [15]:
build36_genotype.head()

,chr,pos,genotype
rsid,,,
rs3094315,1,752566,AA
rs12562034,1,768448,AG
rs3934834,1,1005806,CC
rs9442372,1,1018704,AA
rs3737728,1,1021415,AG
